In [1]:
# Imports and file paths
import sys
import pandas as pd
from datetime import datetime
import os
import json
print('Python version:', sys.version)

# Standard data science libraries
import numpy as np
import pandas as pd
from pathlib import Path

# sklearn & joblib will be used
import sklearn
import joblib
print('pandas', pd.__version__, 'sklearn', sklearn.__version__)

# Paths (these files were uploaded in this session)
TRAIN_CSV = "/content/SENG_4610_Training_Data.csv"
FEATURES_CSV = "/content/Feature_Description.csv"

# Check files
print('Training CSV exists:', Path(TRAIN_CSV).exists())
print('Feature description exists:', Path(FEATURES_CSV).exists())

# Load CSVs
df = pd.read_csv(TRAIN_CSV, encoding='latin-1')
feat = pd.read_csv(FEATURES_CSV, encoding='latin-1')

print('Training data shape:', df.shape)
display(df.head(10))
print('\n---\nFeature description preview:')
display(feat.head(20))

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
pandas 2.2.2 sklearn 1.6.1
Training CSV exists: True
Feature description exists: True
Training data shape: (175341, 45)


,ï»¿id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0
5,6,0.380537,tcp,-,FIN,10,6,534,268,39.417980,...,1,40,0,0,0,2,39,0,Normal,0
6,7,0.637109,tcp,-,FIN,10,8,534,354,26.683033,...,1,40,0,0,0,1,39,0,Normal,0
7,8,0.521584,tcp,-,FIN,10,8,534,354,32.593026,...,1,40,0,0,0,3,39,0,Normal,0
8,9,0.542905,tcp,-,FIN,10,8,534,354,31.313031,...,1,40,0,0,0,3,39,0,Normal,0
9,10,0.258687,tcp,-,FIN,10,6,534,268,57.985135,...,1,40,0,0,0,3,39,0,Normal,0



---
Feature description preview:


,No.,Name,Type,Description
0,1,srcip,nominal,Source IP address
1,2,sport,integer,Source port number
2,3,dstip,nominal,Destination IP address
3,4,dsport,integer,Destination port number
4,5,proto,nominal,Transaction protocol
5,6,state,nominal,Indicates to the state and its dependent proto...
6,7,dur,Float,Record total duration
7,8,sbytes,Integer,Source to destination transaction bytes
8,9,dbytes,Integer,Destination to source transaction bytes
9,10,sttl,Integer,Source to destination time to live value


In [2]:
# Quick EDA
print('Columns:', df.columns.tolist())
print('\nDtypes:\n', df.dtypes.value_counts())

# Missing values summary
missing = df.isna().sum().sort_values(ascending=False)
display(missing.head(30))

# Confirm target column names (adjust if different)
possible_label_cols = ['label', 'Label', 'attack', 'is_attack']
possible_attack_cat = ['attack_cat', 'attack_catagory', 'attack_category', 'attack_type']

print('Columns present that match common names:')
print([c for c in df.columns if c in possible_label_cols])
print([c for c in df.columns if c in possible_attack_cat])

# If your dataset uses different names, update these variables:
BINARY_TARGET = 'label'           # 0 normal, 1 attack
MULTI_TARGET = 'attack_cat'       # multiclass attack category
ID_COL = 'id'

Columns: ['ï»¿id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label']

Dtypes:
 int64      30
float64    11
object      4
Name: count, dtype: int64


,0
ï»¿id,0
dur,0
proto,0
service,0
state,0
spkts,0
dpkts,0
sbytes,0
dbytes,0
rate,0


Columns present that match common names:
['label']
['attack_cat']


In [3]:
# Drop ID if present
if ID_COL in df.columns:
    df = df.drop(columns=[ID_COL])
    print('Dropped id column')

# Clean service & other textual columns
for col in ['service', 'proto', 'state']:
    if col in df.columns:
        df[col] = df[col].astype(str)
        df[col] = df[col].replace(['-', '?', 'None', 'nan', ''], 'unknown')
        df[col] = df[col].fillna('unknown')

# Show unique counts
for col in ['service', 'proto', 'state']:
    if col in df.columns:
        print(f"-> {col}: unique count =", df[col].nunique(), 'example values =', df[col].unique()[:10])

from sklearn.preprocessing import LabelEncoder

# Ensure binary target is numeric 0/1
if df[BINARY_TARGET].dtype == object:
    # if the label is text like 'Normal'/'Attack' or 'normal'/'anomaly'
    df['is_attack'] = df[BINARY_TARGET].str.lower().isin(
        ['1', 'attack', 'anomaly', 'attack!', 'attack?']
    ).astype(int)
else:
    df['is_attack'] = df[BINARY_TARGET].astype(int)

print('is_attack distribution:')
print(df['is_attack'].value_counts())

-> service: unique count = 13 example values = ['unknown' 'ftp' 'smtp' 'snmp' 'http' 'ftp-data' 'dns' 'ssh' 'radius'
 'pop3']
-> proto: unique count = 133 example values = ['tcp' 'udp' 'arp' 'ospf' 'icmp' 'igmp' 'rtp' 'ddp' 'ipv6-frag' 'cftp']
-> state: unique count = 9 example values = ['FIN' 'INT' 'CON' 'ECO' 'REQ' 'RST' 'PAR' 'URN' 'no']
is_attack distribution:
is_attack
1    119341
0     56000
Name: count, dtype: int64


In [4]:
# ============================
# Multiclass encoding (improved)
# ============================

if MULTI_TARGET in df.columns:
    # CHANGED: work on a filtered copy that only keeps rows with a real attack_cat
    df_multi = df[df[MULTI_TARGET].notna()].copy()
    print("\nMulticlass rows (non-NaN attack_cat):", len(df_multi))

    # Optional: show original distribution
    print("\nOriginal attack_cat distribution (non-NaN only):")
    print(df_multi[MULTI_TARGET].value_counts())

    # Label encode attack_cat on the filtered df_multi only
    le_attack = LabelEncoder()
    df_multi['attack_cat_encoded'] = le_attack.fit_transform(
        df_multi[MULTI_TARGET].astype(str)
    )

    print('\nMulticlass mapping:')
    mapping = dict(zip(le_attack.classes_, le_attack.transform(le_attack.classes_)))
    print(mapping)
else:
    df_multi = None
    print('No multiclass attack column found; will skip softmax training unless you update MULTI_TARGET.')

# For sanity, still show attack_cat distribution on the full df (will include NaNs)
if MULTI_TARGET in df.columns:
    print('\nattack_cat value_counts (full df, including NaN):')
    print(df[MULTI_TARGET].value_counts(dropna=False))


Multiclass rows (non-NaN attack_cat): 175341

Original attack_cat distribution (non-NaN only):
attack_cat
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: count, dtype: int64

Multiclass mapping:
{'Analysis': np.int64(0), 'Backdoor': np.int64(1), 'DoS': np.int64(2), 'Exploits': np.int64(3), 'Fuzzers': np.int64(4), 'Generic': np.int64(5), 'Normal': np.int64(6), 'Reconnaissance': np.int64(7), 'Shellcode': np.int64(8), 'Worms': np.int64(9)}

attack_cat value_counts (full df, including NaN):
attack_cat
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: count, dtype: int64


In [5]:
# By default drop IPs and timestamp-like fields to avoid leakage and high-cardinality
drop_cols = []
for cand in ['srcip', 'dstip', 'Stime', 'Ltime', 'stime', 'ltime', 'stime_ms']:
    if cand in df.columns:
        drop_cols.append(cand)

print('Initial drop_cols:', drop_cols)

# Numeric columns detection (from df)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# exclude target columns
exclude = ['is_attack', BINARY_TARGET, 'attack_cat_encoded']
numeric_cols = [c for c in numeric_cols if c not in exclude]
print('\nNumeric candidate columns (sample):', numeric_cols[:20])

# Categorical columns to encode
cat_cols = [c for c in ['service', 'proto', 'state'] if c in df.columns]
print('\nCategorical columns to encode:', cat_cols)

# Final features list — for pipeline we'll handle encoding/scaling
features = [c for c in numeric_cols + cat_cols if c not in drop_cols]
print('\nFinal features to be used (sample):', features[:40])

Initial drop_cols: []

Numeric candidate columns (sample): ['ï»¿id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb']

Categorical columns to encode: ['service', 'proto', 'state']

Final features to be used (sample): ['ï»¿id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports']


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# Define preprocessing steps
num_features = [c for c in features if c in df.columns and np.issubdtype(df[c].dtype, np.number)]
cat_features = [c for c in features if c in df.columns and c not in num_features]

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
], remainder='drop')

print('num_features count:', len(num_features))
print('cat_features count:', len(cat_features))

num_features count: 40
cat_features count: 3


In [7]:
# ============================
# Binary classification
# ============================
from sklearn.neural_network import MLPClassifier

print('\n================ BINARY CLASSIFICATION ================\n')

# Prepare data for binary classification
X = df[features]
y = df['is_attack']

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# Neural network pipeline (same preprocessing)
mlp_model = Pipeline(steps=[
    ('preproc', preprocessor),
    ('clf', MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        max_iter=200,
        random_state=42,
        early_stopping=True,
        verbose=True
    ))
])


================ BINARY CLASSIFICATION ================



In [8]:
print('Training MLPClassifier (binary)...')
mlp_model.fit(X_train, y_train)
print('Done.')



Training MLPClassifier (binary)...
Iteration 1, loss = 0.14329366
Validation score: 0.948318
Iteration 2, loss = 0.10062198
Validation score: 0.955660
Iteration 3, loss = 0.09325353
Validation score: 0.956658
Iteration 4, loss = 0.08567800
Validation score: 0.964357
Iteration 5, loss = 0.07821420
Validation score: 0.967707
Iteration 6, loss = 0.07155414
Validation score: 0.967137
Iteration 7, loss = 0.06597820
Validation score: 0.969276
Iteration 8, loss = 0.06168312
Validation score: 0.972840
Iteration 9, loss = 0.05893828
Validation score: 0.972555
Iteration 10, loss = 0.05705680
Validation score: 0.972840
Iteration 11, loss = 0.05520030
Validation score: 0.972198
Iteration 12, loss = 0.05371209
Validation score: 0.976618
Iteration 13, loss = 0.05292465
Validation score: 0.973339
Iteration 14, loss = 0.05143733
Validation score: 0.972626
Iteration 15, loss = 0.05097852
Validation score: 0.975335
Iteration 16, loss = 0.04912985
Validation score: 0.974907
Iteration 17, loss = 0.0489465

In [9]:
y_pred_nn = mlp_model.predict(X_test)
print('\nClassification report (MLP binary):')
print(classification_report(y_test, y_pred_nn, digits=4))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, y_pred_nn))

# Saving the binary model
joblib.dump(mlp_model, 'mlp_attack_model.joblib')


Classification report (MLP binary):
              precision    recall  f1-score   support

           0     0.9785    0.9685    0.9735     11200
           1     0.9853    0.9900    0.9876     23869

    accuracy                         0.9831     35069
   macro avg     0.9819    0.9793    0.9806     35069
weighted avg     0.9831    0.9831    0.9831     35069


Confusion matrix:
[[10847   353]
 [  238 23631]]


['mlp_attack_model.joblib']

In [10]:
# ============================
# Multiclass classification (attack types)
# ============================
# import tensorflow as tf
# from tensorflow.keras import layers, models, regularizers
# from sklearn.utils.class_weight import compute_class_weight

# print('\n================ MULTICLASS CLASSIFICATION ================\n')

# ============================
# NEW ADVANCED MULTICLASS SECTION
# ============================
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.utils.class_weight import compute_class_weight

print('\n================ MULTICLASS CLASSIFICATION (MODERN MLP) ================\n')

if df_multi is None:
    raise ValueError("df_multi is None; MULTI_TARGET column not found.")

# 1. Setup Data
X_multi = df_multi[features]
y_multi = df_multi['attack_cat_encoded']

# Stratified Split
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multi, y_multi, stratify=y_multi, test_size=0.2, random_state=42
)

# 2. Advanced Preprocessing: Separate Categorical vs Numerical
# We need this separation because we are going to use "Embeddings" for text,
# which works much better than OneHotEncoding for things like 'service' or 'proto'.

cat_cols_to_embed = ['service', 'proto', 'state']
# Filter to ensure these cols actually exist in features
cat_cols_to_embed = [c for c in cat_cols_to_embed if c in X_train_m.columns]
num_cols = [c for c in features if c not in cat_cols_to_embed]

print(f"Embedding Categoricals: {cat_cols_to_embed}")
print(f"Scaling Numericals: {len(num_cols)} columns")

# A. Scale Numericals
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_m[num_cols])
X_test_num = scaler.transform(X_test_m[num_cols])

# B. Encode Categoricals (Ordinal)
# We map strings to integers (0, 1, 2...) so the Neural Net can learn their meaning
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train_cat = ord_enc.fit_transform(X_train_m[cat_cols_to_embed])
X_test_cat = ord_enc.transform(X_test_m[cat_cols_to_embed])

# Handle unknown categories (replace -1 with a new index)
for i in range(X_train_cat.shape[1]):
    max_val = X_train_cat[:, i].max()
    # Replace unknowns (-1) with max_val + 1 so it has its own ID
    fill_val = max_val + 1
    X_train_cat[X_train_cat[:, i] == -1, i] = fill_val
    X_test_cat[X_test_cat[:, i] == -1, i] = fill_val

# 3. Compute Class Weights (To fix imbalance)
classes = np.unique(y_train_m)
weights = compute_class_weight('balanced', classes=classes, y=y_train_m)
class_weights_dict = {int(cls): float(w) for cls, w in zip(classes, weights)}
print("Class weights enabled:", class_weights_dict)




================ MULTICLASS CLASSIFICATION (MODERN MLP) ================

Embedding Categoricals: ['service', 'proto', 'state']
Scaling Numericals: 40 columns
Class weights enabled: {0: 8.767, 1: 10.040944881889764, 2: 1.4297421261848946, 3: 0.5250879688552819, 4: 0.9642675465731766, 5: 0.43835, 6: 0.31310714285714286, 7: 1.6712975098296199, 8: 15.482560706401767, 9: 134.87692307692308}


In [11]:
# ==========================================
# REPLACE YOUR MODEL DEFINITION WITH THIS MOBILE-CNN
# ==========================================

# def build_mobile_cnn(num_input_shape, cat_input_shapes, cat_counts, num_classes):
#     # --- 1. Inputs ---
#     input_num = layers.Input(shape=num_input_shape, name="input_num")

#     inputs_cat = []
#     embeddings = []

#     # Use smaller embeddings for mobile (output_dim=8 is usually enough)
#     for i, (col_name, count) in enumerate(zip(cat_cols_to_embed, cat_counts)):
#         inp = layers.Input(shape=(1,), name=f"input_{col_name}")
#         inputs_cat.append(inp)

#         # Squeeze embedding size for memory efficiency
#         emb_dim = min(16, int(count/2) + 1)
#         emb = layers.Embedding(input_dim=int(count)+2, output_dim=emb_dim)(inp)
#         emb = layers.Flatten()(emb)
#         embeddings.append(emb)

#     # --- 2. Feature Fusion ---
#     # Combine everything into one flat vector
#     x = layers.Concatenate()(embeddings + [input_num])

#     # --- 3. Reshape for CNN ---
#     # The CNN needs a 3D input: (Batch, Steps, Channels)
#     # We treat the concatenated features as a "time series" of length 1, with N channels
#     # Or more effectively for 1D CNN: Reshape to (Features, 1)
#     x = layers.Reshape((-1, 1))(x)

#     # --- 4. Mobile-Optimized CNN Block ---

#     # Conv Block 1
#     x = layers.Conv1D(filters=64, kernel_size=3, padding='same')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('swish')(x)
#     x = layers.MaxPooling1D(pool_size=2)(x)

#     # Conv Block 2
#     x = layers.Conv1D(filters=128, kernel_size=3, padding='same')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('swish')(x)
#     # Global Average Pooling is the key for mobile:
#     # It replaces the massive "Flatten + Dense" layers that eat up memory.
#     x = layers.GlobalAveragePooling1D()(x)

#     # --- 5. Output Head ---
#     x = layers.Dropout(0.2)(x)
#     outputs = layers.Dense(num_classes, activation='softmax')(x)

#     model = models.Model(inputs=inputs_cat + [input_num], outputs=outputs)
#     return model

# # Setup inputs
# cat_counts = [int(X_train_cat[:, i].max()) + 1 for i in range(len(cat_cols_to_embed))]
# num_classes = len(np.unique(y_train_m))
# ==========================================
# REPLACE WITH THIS HYBRID MLP
# ==========================================

def build_hybrid_mlp(num_input_shape, cat_input_shapes, cat_counts, num_classes):
    # --- 1. Inputs ---
    input_num = layers.Input(shape=num_input_shape, name="input_num")

    inputs_cat = []
    embeddings = []

    # Keep embeddings small for mobile efficiency
    for i, (col_name, count) in enumerate(zip(cat_cols_to_embed, cat_counts)):
        inp = layers.Input(shape=(1,), name=f"input_{col_name}")
        inputs_cat.append(inp)

        emb_dim = min(10, int(count/2) + 1) # Small efficient embeddings
        emb = layers.Embedding(input_dim=int(count)+2, output_dim=emb_dim)(inp)
        emb = layers.Flatten()(emb)
        embeddings.append(emb)

    # --- 2. Feature Fusion ---
    # Concatenate everything (Numerics + Embeddings)
    x = layers.Concatenate()(embeddings + [input_num])

    # --- 3. Optimized Dense Block (The "Brain") ---
    # We use "Swish" activation and BatchNormalization to speed up learning

    # Layer 1 (512 neurons)
    x = layers.Dense(512)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    x = layers.Dropout(0.4)(x) # Higher dropout because model is bigger

    # Layer 2 (256 neurons)
    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    x = layers.Dropout(0.3)(x)

    # Layer 3 (128 neurons)
    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    x = layers.Dropout(0.2)(x)

    # --- 4. Output ---
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs_cat + [input_num], outputs=outputs)
    return model

# Setup inputs
cat_counts = [int(X_train_cat[:, i].max()) + 1 for i in range(len(cat_cols_to_embed))]
num_classes = len(np.unique(y_train_m))

In [12]:
# # Build the Mobile CNN
# model = build_mobile_cnn(
#     num_input_shape=(X_train_num.shape[1],),
#     cat_input_shapes=[(1,) for _ in cat_cols_to_embed],
#     cat_counts=cat_counts,
#     num_classes=num_classes
# )

# # Compile (Note: Lower learning rate for CNNs usually helps stability)
# model.compile(
#     optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-4),
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# model.summary()

# Build
model = build_hybrid_mlp(
    num_input_shape=(X_train_num.shape[1],),
    cat_input_shapes=[(1,) for _ in cat_cols_to_embed],
    cat_counts=cat_counts,
    num_classes=num_classes
)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_service       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_proto         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_state         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 7)      │        105 │ input_service[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 10)     │      1,350 │ input_proto[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 5)      │         55 │ input_state[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 7)         │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 10)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 5)         │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_num           │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 62)        │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ flatten_2[0][0],  │
│                     │                   │            │ input_num[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │     32,256 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 512)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256)       │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                 

 Total params: 202,864 (792.44 KB)

 Trainable params: 201,072 (785.44 KB)

 Non-trainable params: 1,792 (7.00 KB)

In [ ]:
# # 5. Train the Model
# We must prepare inputs as a list: [Cat1, Cat2, Cat3, ..., Numericals]
train_inputs = [X_train_cat[:, i] for i in range(X_train_cat.shape[1])] + [X_train_num]
test_inputs = [X_test_cat[:, i] for i in range(X_test_cat.shape[1])] + [X_test_num]

# history = model.fit(
#     train_inputs,
#     y_train_m,
#     epochs=100,
#     batch_size=1024,
#     # class_weight=class_weights_dict, # CRITICAL: Handles the imbalance
#     validation_split=0.2,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
#         tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
#     ],
#     verbose=1
# )

history = model.fit(
    train_inputs,
    y_train_m,
    epochs=100,
    batch_size=512,  # 512 is stable for MLPs
    # class_weight=class_weights_dict,  <-- KEEP COMMENTED OUT for max accuracy
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
    ],
    verbose=1
)

Epoch 1/100
220/220 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.6885 - loss: 0.9212 - val_accuracy: 0.7864 - val_loss: 0.5851 - learning_rate: 0.0010
Epoch 2/100
220/220 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.7792 - loss: 0.5889 - val_accuracy: 0.7967 - val_loss: 0.5382 - learning_rate: 0.0010
Epoch 3/100
220/220 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - accuracy: 0.7870 - loss: 0.5603 - val_accuracy: 0.7984 - val_loss: 0.5251 - learning_rate: 0.0010
Epoch 4/100
220/220 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.7922 - loss: 0.5468 - val_accuracy: 0.8063 - val_loss: 0.5122 - learning_rate: 0.0010
Epoch 5/100
220/220 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.7957 - loss: 0.5376 - val_accuracy: 0.8011 - val_loss: 0.5044 - learning_rate: 0.0010
Epoch 6/100
220/220 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.7981 - loss: 0.5288 - val_accuracy: 0.8063 - val_loss: 0.4958 - learning_rate: 0.0010
Epoch 7/100
220/220 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.7996 

In [ ]:
import matplotlib.pyplot as plt

# accuracy
plt.figure(figsize=(6,4))
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title("Accuracy vs Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

# loss
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title("Loss vs Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()


# If train accuracy shoots up but val accuracy flattens or drops, your model is
# too big / too complex → reduce layers, reduce units, or increase dropout/L2.

# If both curves move up together → you're safe.

In [ ]:
model.save("multiclass_model.h5")

import os
size_mb = os.path.getsize("multiclass_model.h5") / (1024*1024)
print(f"Model size: {size_mb:.2f} MB")

from google.colab import files
files.download("multiclass_model.h5")

In [ ]:
# Predictions
y_pred_probs = model.predict(X_test_num)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

print('\nClassification report (Keras multiclass):')
print(classification_report(y_test_m, y_pred_classes, digits=4))
print('\nConfusion matrix (Keras multiclass):')
print(confusion_matrix(y_test_m, y_pred_classes))
